In [101]:
import smbus, time
import numpy as np
i2c= smbus.SMBus(1)

class MPU9250():
    def __init__(self): 
        print('Programming MPU9250...')
        
        i2c.write_byte_data(0x68, 0x37, 0x02) # set bypass mode for magnetometer
        
        # configure accelerometer
        #self.accel_sens=2*9.80665/32768; i2c.write_byte_data(0x68, 0x1C,0b00000000) #LSB to m/s2
        self.accel_sens=4*9.80665/32768; i2c.write_byte_data(0x68, 0x1C,0b00001000)
        #self.accel_sens=8*9.80665/32768; i2c.write_byte_data(0x68, 0x1C,0b00010000)
        #self.accel_sens=16*9.80665/32768; i2c.write_byte_data(0x68, 0x1C,0b00011000)
        
        # configure gyroscope
        self.gyro_sens= 1/131; i2c.write_byte_data(0x68, 0x1B,0b00000000) #LSB to deg/s
        #self.gyro_sens=1/65.5; i2c.write_byte_data(0x68, 0x1B,0b00001000)
        #self.gyro_sens=1/32.8; i2c.write_byte_data(0x68, 0x1B,0b00010000)
        #self.gyro_sens=1/16.4; i2c.write_byte_data(0x68, 0x1B,0b00011000)
        
        # configure magnetometer
        self.magnet_sens=0.6e-6*1e4 #LSB to gauss
        
        # configure temperature
        self.temp_sens=1/333.87 #LSB  to deg celsius
        self.temp_offset=21 #deg celsius <!> note sign
        print('Done')
        
        print('Initializing IMU: Make sure IMU is in idle...')
        self.gyro_offset=0;
        self.gyro_offset=self.readGyro() #deg/s
        print('Done')   
        
    def readWord(self, address, high, low): 
        return float(np.int16((i2c.read_byte_data(address, high) << 8)|i2c.read_byte_data(address, low)))
    
    def readAccel(self):
        accel_x = self.readWord(0x68, 0x3B, 0x3C)
        accel_y = self.readWord(0x68, 0x3D, 0x3E)
        accel_z = self.readWord(0x68, 0x3F, 0x40)
        return np.multiply(self.accel_sens,[accel_x, accel_y, accel_z])
    
    def readGyro(self):
        gyro_x = self.readWord(0x68, 0x43, 0x44)
        gyro_y = self.readWord(0x68, 0x45, 0x46)
        gyro_z = self.readWord(0x68, 0x47, 0x48)   
        return np.subtract(np.multiply(self.gyro_sens,[gyro_x,gyro_y,gyro_z]),self.gyro_offset)
    
    def readTemp(self): 
        temp = self.readWord(0x68, 0x41, 0x42)
        return self.temp_sens*temp+self.temp_offset
    
    def readMagnet(self):    
        # MPU9250 datsheet page 24: slave address for the AK8963 is 0X0C
        i2c.write_byte_data(0x0C, 0x0A, 0x12)
        time.sleep(0.15)
        # calculate
        magnet_x = self.readWord(0x0C, 0x04, 0x03)
        magnet_y = self.readWord(0x0C, 0x06, 0x05)
        magnet_z = self.readWord(0x0C, 0x08, 0x07)
        return np.multiply(self.magnet_sens,[magnet_x, magnet_y, magnet_z])



class BMP280():
    def __init__(self):
        print('Programming BMP280...')
        i2c.write_byte_data(119, 0xF4,0b00100111)# enable pressure and temperature sensors
        self.dig_T1= np.uint16(self.readWord(0x77, 0x89, 0x88))
        self.dig_T2= self.readWord(0x77, 0x8B, 0x8A)
        self.dig_T3= self.readWord(0x77, 0x8D, 0x8C)
        self.dig_P1 = np.uint16(self.readWord(0x77, 0x8F, 0x8E))
        self.dig_P2 = self.readWord(0x77, 0x91, 0x90)
        self.dig_P3 = self.readWord(0x77, 0x93, 0x92)
        self.dig_P4 = self.readWord(0x77, 0x95, 0x94)
        self.dig_P5 = self.readWord(0x77, 0x97, 0x96)
        self.dig_P6 = self.readWord(0x77, 0x99, 0x98)
        self.dig_P7 = self.readWord(0x77, 0x9B, 0x9A)
        self.dig_P8 = self.readWord(0x77, 0x9D, 0x9C)
        self.dig_P9 = self.readWord(0x77, 0x9F, 0x9E)
  
    def readWord(self, address, high, low): 
        return np.int16((i2c.read_byte_data(address, high) << 8)|i2c.read_byte_data(address, low))
    
    def readTemp(self):
        return ((self.readTempFine() * 5 + 128)>> 8)/100
    
    def readTempFine(self): 
        temp_raw =((i2c.read_byte_data(0x77, 0xFA) << 8)|i2c.read_byte_data(0x77, 0xFB))<<4|((i2c.read_byte_data(0x77, 0xFC)>>4)&0xF)
        var1= ((((temp_raw>>3) - (self.dig_T1<<1)))*(self.dig_T2)) >> 11
        var2= (((((temp_raw>>4) - (self.dig_T1)) * ((temp_raw>>4)-(self.dig_T1))) >> 12) *(self.dig_T3)) >> 14;
        return var1+var2
    
    def readPressure(self):    
        pres_raw = ((i2c.read_byte_data(0x77, 0xF7) << 8)|i2c.read_byte_data(0x77, 0xF8))<<4|((i2c.read_byte_data(0x77, 0xF9)>>4)&0xF)
        var1 = self.readTempFine()/2 - 64000;
        var2 = var1 * var1 * self.dig_P6/32768.0;
        var2 = var2 + (var1*self.dig_P5)*2;
        var2 = var2/4 + self.dig_P4*65536.0;
        var1 = (self.dig_P3 * var1 * var1 / 524288.0 + self.dig_P2 * var1) / 524288.0
        var1 = (1.0 + var1 / 32768.0) * self.dig_P1
        pres = 1048576.0-pres_raw;
        pres = (pres - var2 / 4096.0) * 6250.0 / var1
        var1 = self.dig_P9 * pres * pres / 2147483648.0
        var2 = pres * self.dig_P8 / 32768.0
        return pres + (var1 + var2 + self.dig_P7) / 16.0
if __name__ == '__main__':
    mpu9250 = MPU9250()
    bmp280 = BMP280()
    for i in range(10):
        print("-----------------------------------")
        print("Accelerations [m/s²]=",mpu9250.readAccel())
        print("Angular Velocities [deg/s]=",mpu9250.readGyro())
        print("Magnetic Field [G]=",mpu9250.readMagnet())
        print("Temperatures [°C]=",[mpu9250.readTemp(),bmp280.readTemp()])
        print("Pressure[Pa]=",bmp280.readPressure())  
        time.sleep(0.2)



Programming MPU9250...
Done
Initializing IMU: Make sure IMU is in idle...
Done
Programming BMP280...
-----------------------------------
Accelerations [m/s²]= [ 1.13485159  9.88805286 -0.98641108]
Angular Velocities [deg/s]= [-0.25954198 -0.08396947  0.14503817]
Magnetic Field [G]= [-2.064 -0.504 -3.612]
Temperatures [°C]= [30.392877467277685, 25.47]
Pressure[Pa]= 96782.15370566814
-----------------------------------
Accelerations [m/s²]= [ 1.06541974  9.75637177 -1.06781394]
Angular Velocities [deg/s]= [-1.75572519 -1.76335878 -0.32824427]
Magnetic Field [G]= [-1.962 -0.534 -3.558]
Temperatures [°C]= [30.728337376823315, 25.73]
Pressure[Pa]= 96778.86694601091
-----------------------------------
Accelerations [m/s²]= [ 1.07260234  9.55525883 -0.92655604]
Angular Velocities [deg/s]= [-0.13740458  0.21374046  0.16030534]
Magnetic Field [G]= [-1.98  -0.444 -3.576]
Temperatures [°C]= [31.447180040135382, 26.03]
Pressure[Pa]= 96790.91870074552
-----------------------------------
Acceleratio